In [ ]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time

import Init_NetRate as Init
import Cascade_generation_functions_NetRate as Gen
import CVX_functions as cvx 

In [ ]:
network_file_name = "./Graph_test_SG.txt"
cascade_file_name = "./Cascade_test_SG.txt"
window = 10
beta = 0.5
model = 0

In [ ]:
# G_true = Gen.Generate_random_graph(400,800)
# Cascades = Gen.Generate_all_cascades(G_true,-2000,window,model,beta)
# Gen.Save_cascade_to_file("Cascade_test.txt",Cascades,G_true)
# Gen.Save_graph_to_file("Graph_test.txt",G_true)
# G_star,DAG_C_dic = Init.Init("Cascade_test.txt")
G_true = Init.Load_ground_truth(network_file_name)
G_star, DAG_C_dic = Init.Init(cascade_file_name)

In [ ]:
s_time = time.time()
A_list,nb_c_per_node = cvx.Create_matrices(G_true,DAG_C_dic,window)
alpha_mat,final_obj_value = cvx.Infer_Network_edges(G_true,A_list,nb_c_per_node,DAG_C_dic)
print("final_obj_value :",final_obj_value)
f_time = time.time()
elapsed_time = f_time-s_time
print(elapsed_time)

In [ ]:
'''
Precision and recall computation
'''
correct = 0
edge_G_star = list(G_star.edges())
G_star.remove_edges_from(edge_G_star)
for i in range(0,G_true.number_of_nodes()):
    for j in range(0,G_true.number_of_nodes()):
        if alpha_mat[i,j] >=1e-2:
            G_star.add_edge(i,j)
            if (i,j) in G_true.edges():
                correct +=1
#                 print("edge ",(i,j))
#                 print("alpha is ",A_hat[i,j])
print("Precision :",correct/G_star.number_of_edges())
print("Recall :",correct/G_true.number_of_edges())
print("Number of correct infered edges : ",correct)
print("Number of edges in G_hat :",G_star.number_of_edges())
print("Number of edges in the true network ",G_true.number_of_edges())

In [ ]:
'''
mean absolute error
'''
mae = 0
nb_correcte_edges=0
for edge in G_true.edges():
    if edge in G_star.edges():
        nb_correcte_edges +=1
        true_alpha = G_true.edges[edge]["weight"][0]
        approx_alpha = alpha_mat[edge[0],edge[1]]
        mae += abs(true_alpha-approx_alpha)/true_alpha #normalized mean absolute error
#         print("For edge " + str(edge)+ " the true weight is : " + str(true_alpha))
#         print("For edge " + str(edge)+ " the approx weight is : " + str(approx_alpha))
try :
    mae = mae/nb_correcte_edges
except ZeroDivisionError:
    print ("There is no correct edge")
print("Normalized mean absolute error is : ",mae)